# Viewing angles

Visualizing the illumination and viewing angles. Explaining how ViSiProg groups are related?

In [11]:
%load_ext autoreload
%autoreload 2
import networkx as nx
import numpy as np
import os

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import plotly.graph_objs as go
from visiprog.data import read_VSP_label, read_img_list, read_viewing_conditions, viewing_condition_index
from visiprog.data import read_viewing_conditions, polar_to_euclidean, angle_between
from IPython.display import display, HTML

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df_viewing = read_viewing_conditions()
print(df_viewing.shape)

(205, 4)


In [3]:
def make_image_html(image_path):
     return '<img src="{}" width="100" height="100" style="display:inline;margin:1px"/>'.format(image_path)

In [6]:
groups = read_VSP_label()
list_img = read_img_list()

traces = []

group = groups[100]

for i in group: 
    viewing_index = viewing_condition_index(list_img[i])
    
    illum_x, illum_y, illum_z = polar_to_euclidean(df_viewing['illum_theta'][viewing_index],df_viewing['illum_phi'][viewing_index])
    view_x, view_y, view_z = polar_to_euclidean(df_viewing['view_theta'][viewing_index],df_viewing['view_phi'][viewing_index])
    
    line_x = [illum_x, illum_x, 0, view_x]
    line_y = [illum_y, illum_y, 0, view_y]
    line_z = [illum_z - 0.05,illum_z, 0, view_z]
    
    # plotting line
    line_scatter = go.Scatter3d(
        x=line_x,
        y=line_y,
        z=line_z,
        name=os.path.basename(list_img[i])
    )
    
    traces.append(line_scatter)

layout = go.Layout(
    title='Angles of VSP group',
    autosize=True,
    width=900,
    height=800,
    margin=dict(
        l=65,
        r=50,
        b=65,
        t=90
    )
)
fig = go.Figure(data=traces, layout=layout)
iplot(fig, filename='vspgroup')

# display the images as well
display(HTML(''.join(make_image_html(list_img[i]) for i in group) + '<br>' + \
            '\t'.join(os.path.basename(list_img[i]) for i in group)))

# Formal analysis using graph and minimum spanning tree

In [16]:
groups = read_VSP_label(pappas_only=False)
list_img = read_img_list()

print(len(groups))

mst_distances = []

for group in groups:
    
    print("=================================================")
    
    print(group)
    
    # building the weighted undirected graph
    G = nx.Graph()
    G.add_nodes_from(range(len(group)))

    for i in range(len(group)):
        index_i = viewing_condition_index(list_img[group[i]])
        
        illum_x_1, illum_y_1, illum_z_1 = polar_to_euclidean(df_viewing['illum_theta'][index_i],df_viewing['illum_phi'][index_i])
        view_x_1, view_y_1, view_z_1 = polar_to_euclidean(df_viewing['view_theta'][index_i],df_viewing['view_phi'][index_i])

        
        for j in range(i + 1, len(group)):
            index_j = viewing_condition_index(list_img[group[j]])
            illum_x_2, illum_y_2, illum_z_2 = polar_to_euclidean(df_viewing['illum_theta'][index_j],df_viewing['illum_phi'][index_j])
            view_x_2, view_y_2, view_z_2 = polar_to_euclidean(df_viewing['view_theta'][index_j],df_viewing['view_phi'][index_j])
            
            weight_illum = angle_between((illum_x_1, illum_y_1, illum_z_1),(illum_x_2, illum_y_2, illum_z_2))
            weight_view = angle_between((view_x_1, view_y_1, view_z_1),(view_x_2, view_y_2, view_z_2))
        
            G.add_edge(i, j, weight=weight_illum)
            G.add_edge(i, j, weight=weight_view)
            
#     print(G.edges(data=True))
#     print("\n\nSorting the MST\n\n")
    
    # minimum spanning tree
    mst=nx.minimum_spanning_edges(G,data=True)
    
    distances = []
    for e in mst:
        distances.append(e[2]['weight'])
        
    mst_distances.append(distances)

604
[1789, 1787, 1775, 1778, 1782, 1828, 1830, 1781, 1811]
[4357, 4355, 4374, 4345, 4347, 4376, 4354, 4349, 4339]
[2635, 981, 1007, 980, 1042, 2609, 2655, 978, 1004]
[4283, 4305, 4278, 4281, 4250, 4248, 4251, 4249, 4302]
[3599, 3630, 3597, 3633, 3598, 3676, 3629, 3655, 3634]
[1606, 1629, 1626, 1630, 1648, 1595, 1649, 1628, 1662]
[1973, 1970, 1968, 1961, 1972, 1964, 1963, 1969, 1971]
[4898, 4892, 4894, 4923, 4895, 4905, 4907, 4930, 4900]
[4801, 4870, 4854, 4852, 4850, 4828, 4831, 4794, 4826]
[4564, 4532, 4541, 4527, 4535, 4524, 4554, 4540, 4557]
[2351, 2349, 2372, 2347, 2374, 2317, 2373, 2371, 2316]
[1121, 1070, 1099, 1072, 1069, 1119, 1097, 1059, 1098]
[3066, 3112, 3098, 3129, 3061, 3065, 3094, 3068, 3113]
[2270, 2240, 2269, 2242, 2247, 2263, 2236, 2285, 2266]
[4076, 4069, 4077, 4070, 4068, 4098, 4067, 4100, 4122]
[1537, 1511, 1499, 1507, 1497, 1570, 1509, 1508, 1552]
[819, 817, 842, 789, 790, 820, 822, 786, 818]
[3339, 3345, 3341, 3335, 3363, 3372, 3369, 3340, 3368]
[1314, 1295, 1280,

ValueError: invalid literal for int() with base 10: '128m'

In [17]:
mst_out = np.asarray(mst_distances)
np.savetxt('mst_distances.csv', mst_out, fmt='%.4f', delimiter=',')

In [18]:
print(mst_out.shape)

(77, 8)
